In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import keras.backend as K

Using TensorFlow backend.


In [2]:
(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()
X_train, X_test = X_train/255, X_test/255
if len(X_train.shape) != 4:
    X_train, X_test = X_train[:,:,:,np.newaxis], X_test[:,:,:,np.newaxis]

In [3]:
class FrequencyMultiplicative(keras.layers.Layer):
    
    def __init__(self, filters, **kwargs):
        self.filters = filters
        super(FrequencyMultiplicative, self).__init__(**kwargs)
    
    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(1, self.filters, input_shape[3], input_shape[1], input_shape[2]),
            initializer='he_uniform', name='kernel')
        self.bias = self.add_weight(
            shape=(self.filters,),
            initializer='zeros', name='bias')
        super(FrequencyMultiplicative, self).build(input_shape)
    
    def call(self, inputs):
        x = inputs
        x = K.permute_dimensions(x, (0,3,2,1))
        x = tf.spectral.dct(x, norm='ortho')
        x = K.permute_dimensions(x, (0,1,3,2))
        x = tf.spectral.dct(x, norm='ortho')
        x = K.permute_dimensions(x, (0,1,3,2))
        x = K.expand_dims(x, axis=1)
        x = x * self.kernel
        x = K.sum(x, axis=2, keepdims=False)
        x = K.permute_dimensions(x, (0,1,3,2))
        x = tf.spectral.idct(x, norm='ortho')
        x = K.permute_dimensions(x, (0,1,3,2))
        x = tf.spectral.idct(x, norm='ortho')
        x = K.permute_dimensions(x, (0,2,3,1))
        x = x + self.bias
        return x
    
    def compute_output_shape(self, input_shape):
        return input_shape[:-1] + (self.filters,)

In [4]:
X = X_input = keras.layers.Input((28, 28, 1))
X = keras.layers.BatchNormalization()(X)
X = FrequencyMultiplicative(8)(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Activation('relu')(X)
X = keras.layers.AveragePooling2D()(X)
X = FrequencyMultiplicative(16)(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Activation('relu')(X)
X = keras.layers.AveragePooling2D()(X)
X = FrequencyMultiplicative(32)(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Activation('relu')(X)
X = keras.layers.Flatten()(X)
X = keras.layers.Dense(10, activation='softmax')(X)
M = keras.Model(X_input, X)
M.compile('nadam', 'sparse_categorical_crossentropy', ['acc'])
M.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 1)         4         
_________________________________________________________________
frequency_multiplicative_1 ( (None, 28, 28, 8)         6280      
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 8)         32        
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 8)         0         
_________________________________________________________________
average_pooling2d_1 (Average (None, 14, 14, 8)         0         
_________________________________________________________________
frequency_multiplicative_2 ( (None, 14, 14, 16)        25104     
__________

In [5]:
datagen = keras.preprocessing.image.ImageDataGenerator(
    shear_range=0.1,
    zoom_range=0.1,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1)
datagen.fit(X_train)

In [6]:
M.fit_generator(
    datagen.flow(X_train, Y_train, batch_size=64), 
    validation_data=(X_test, Y_test),
    steps_per_epoch=len(X_train) / 64, epochs=30, callbacks=[
    keras.callbacks.ReduceLROnPlateau(patience=3, verbose=1),
])

Epoch 1/30
938/937 [==============================] - 65s 70ms/step - loss: 0.2345 - acc: 0.9270 - val_loss: 0.1758 - val_acc: 0.9406
Epoch 2/30
938/937 [==============================] - 63s 68ms/step - loss: 0.1150 - acc: 0.9644 - val_loss: 0.0605 - val_acc: 0.9790
Epoch 3/30
938/937 [==============================] - 63s 67ms/step - loss: 0.0892 - acc: 0.9721 - val_loss: 0.0462 - val_acc: 0.9839
Epoch 4/30
938/937 [==============================] - 65s 69ms/step - loss: 0.0755 - acc: 0.9761 - val_loss: 0.0473 - val_acc: 0.9839
Epoch 5/30
938/937 [==============================] - 64s 68ms/step - loss: 0.0649 - acc: 0.9786 - val_loss: 0.0416 - val_acc: 0.9863
Epoch 6/30
938/937 [==============================] - 65s 69ms/step - loss: 0.0587 - acc: 0.9806 - val_loss: 0.0326 - val_acc: 0.9882
Epoch 7/30
938/937 [==============================] - 64s 68ms/step - loss: 0.0556 - acc: 0.9829 - val_loss: 0.0334 - val_acc: 0.9895
Epoch 8/30
938/937 [==============================] - 64s 68ms